<a href="https://colab.research.google.com/github/bimbomuri/FaceMaskDetector/blob/master/Face_mask.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import numpy as np
from glob import glob
import os
from sklearn.preprocessing import LabelBinarizer
from sklearn.model_selection import train_test_split
from keras.optimizers import SGD

Using TensorFlow backend.


In [0]:
import cv2
from PIL import ImageFile
import matplotlib.pyplot as plt
%matplotlib inline
import keras
from keras.models import Sequential, Model
from keras.layers import Dense, Dropout, Activation, Flatten,BatchNormalization
from keras.layers import Convolution2D, MaxPooling2D


In [0]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [0]:

imagepath = list(glob("/content/drive/My Drive/Data/*/*"))

# print number of images in the dataset
print('There are %d total images.' % len(imagepath))


There are 1376 total images.


In [0]:
data = []
labels = []

In [0]:
# loop over the image paths
for img in imagepath:
  # extract the class label from the filename
  label = img.split(os.path.sep)[-2]
  # load the input image (224x224) and preprocess it
  image = cv2.imread(img)
  image = cv2.resize(image, (224,224))/255.0
 # update the data and labels lists, respectively
  data.append(image)
  labels.append(label)



In [0]:
# convert the data and labels to NumPy arrays
data = np.array(data,dtype='float32')
labels = np.array(labels)

In [0]:
# perform one-hot encoding on the labels
lb = LabelBinarizer()
labels = lb.fit_transform(labels)
labels = keras.utils.to_categorical(labels)

In [0]:
X_train, X_test, y_train, y_test = train_test_split(data,labels, test_size=0.25,random_state=101)

In [0]:
X_valid, X_test, y_valid, y_test = train_test_split(data, labels, test_size=0.5,random_state=101)

In [0]:
X_train.shape

(1032, 224, 224, 3)

In [0]:
X_test.shape

(688, 224, 224, 3)

In [0]:
X_valid.shape

(688, 224, 224, 3)

In [0]:
from keras.applications.resnet50 import ResNet50

from keras.layers import GlobalAveragePooling2D
from keras.applications import InceptionV3

base_model = InceptionV3(weights='imagenet', include_top=False)

FMModel = base_model.output
FMModel = GlobalAveragePooling2D()(FMModel)
# let's add a fully-connected layer
FMModel = Dense(128, activation='relu')(FMModel)
# and a logistic layer -- let's say we have 200 classes
FMModel = Dropout(0.5)(FMModel)
FMModel = Dense(2, activation='softmax')(FMModel)

# this is the model we will train
model = Model(inputs=base_model.input, outputs=FMModel)

# first: train only the top layers (which were randomly initialized)
# i.e. freeze all convolutional InceptionV3 layers
for layer in base_model.layers:
    layer.trainable = False



In [0]:
FMModel = model

In [0]:
sgd = SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)
model.compile(loss='categorical_crossentropy',
                           optimizer=sgd,
                           metrics=['accuracy'])

In [0]:
model.fit(X_train, y_train, batch_size=32,
                           epochs=20,
                           validation_data=(X_valid,y_valid),shuffle=True)

Train on 1032 samples, validate on 688 samples
Epoch 1/20
1032/1032 [==============================] - 204s 198ms/step - loss: 0.4116 - accuracy: 0.7878 - val_loss: 0.0770 - val_accuracy: 0.9782
Epoch 2/20
1032/1032 [==============================] - 200s 194ms/step - loss: 0.1542 - accuracy: 0.9448 - val_loss: 0.1025 - val_accuracy: 0.9695
Epoch 3/20
1032/1032 [==============================] - 203s 196ms/step - loss: 0.1301 - accuracy: 0.9496 - val_loss: 0.0685 - val_accuracy: 0.9797
Epoch 4/20
1032/1032 [==============================] - 200s 193ms/step - loss: 0.1068 - accuracy: 0.9622 - val_loss: 0.0653 - val_accuracy: 0.9826
Epoch 5/20
1032/1032 [==============================] - 200s 194ms/step - loss: 0.1018 - accuracy: 0.9651 - val_loss: 0.0558 - val_accuracy: 0.9855
Epoch 6/20
1032/1032 [==============================] - 200s 193ms/step - loss: 0.0750 - accuracy: 0.9690 - val_loss: 0.0768 - val_accuracy: 0.9840
Epoch 7/20
1032/1032 [==============================] - 200s 194m

In [0]:
y_pred=model.predict(X_test)

In [0]:
y_pred = y_pred.argmax(axis=1) 

In [0]:
from sklearn.metrics import classification_report,accuracy_score

In [0]:
print(classification_report(y_test.argmax(axis=1), y_pred))

              precision    recall  f1-score   support

           0       1.00      0.99      0.99       348
           1       0.99      1.00      0.99       340

    accuracy                           0.99       688
   macro avg       0.99      0.99      0.99       688
weighted avg       0.99      0.99      0.99       688



In [0]:
print(accuracy_score(y_test.argmax(axis=1), y_pred))

0.9927325581395349
